## Identifying inefficient operators

In the course of this study, we will analyze the data of call centers provided by the operator of virtual telephony "Nupozvoni".

**The purpose of the investigation**

Prepare a study of call center data and find inefficient operators

**The essence of research**

To conduct data analysis, the results of which will help to build further work with call center employees

**Research objectives**

Perform preprocessing of the initial data - fill in the gaps, work out anomalies and analyze the half-formed data array to answer the questions posed in the purpose of the study

**Initial data**

Two tables are presented for the study. One of the tables contains data on call center clients. The table includes the tariff plan and the date of registration in the service. There are 732 entries in total. The other is with information about calls, containing 53902 records and including the date, direction, type of call, operator id, number of calls, etc.

**This study will be divided into several parts**
1. Data overview
2. Data preprocessing:
 - bringing the data to the required format
 - search for explicit and implicit duplicates
 - filling in gaps
3. Data analysis:
 - distribution of call centers
 - identification of inefficient operators
4. Hypothesis testing
5. Conclusions

The distribution of call centers will go into 2 categories - specializing in incoming and outgoing calls. The division will go by 50% - if there are more than 50% outgoing calls in the call center, then this center specializes in outgoing calls

How to identify an inefficient operator:
1. By the average number of missed calls per day. It is usually assumed that when the operator is fully loaded, he should allow no more than 1-2 missed calls per day. As a standard, the operator should not miss calls, but the situations may be different. The client may call and quickly reset the handset, or a technical error may occur. Therefore, there will always be a certain number of missed calls regardless of the efficiency of the operator. 1-2 missed calls are at full load, which implies about 100 calls per day. But the load is different, so it's better to count as a percentage. Let's assume that if an operator has on average more than 2% of missed calls, such an operator is ineffective.
2. By waiting time for the operator's response. If the average response waiting time is more than 28 seconds, then the operator is ineffective. The data is based on the report https://www.ifc.org/wps/wcm/connect/3a656e01-ad18-459a-b1ed-87784c1f1616/Tool+9.4.+Measuring+Call+Center+Performance.pdf?MOD=AJPERES&CVID=jENLHxM. We do not have waiting data on the operator's phone, only the total waiting time, which depends, among other things, on the total line load at the moment. Therefore, this metric shows rather not the inefficiency of a particular operator, but that the call center or the system of work is ineffective.
3. In this case, the metric for outgoing and incoming call centers will be different. Call centers for incoming calls must have a reserve of free operators to handle calls during peak hours.Let's set the following gradation - if the conversation time of a call center employee working with incoming calls is less than 3 hours a day, then we consider such an operator ineffective. If with outgoing - 4 hours of work per day. The figures are based on the data of the report from the previous paragraph. According to him, the average workload should be at the level of 60-80%, the average talk time is 4 minutes, post-processing is 6 minutes.

Let's form a hypothesis. As a hypothesis, let's compare the average number of inefficient employees in call centers, depending on the tariff plan.
Accordingly, 3 hypotheses will be tested:
1. The duration of outgoing and incoming calls differs
2. There are fewer missed internal calls than client calls
3. The number of inefficient employees in call centers at tariff A is greater than at tariff B

### Data overview

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore") 

try:
    tariff = pd.read_csv('D:/Документы/LinkedIn/9 Выпускной проект/telecom_clients.csv')
    data = pd.read_csv('D:/Документы/LinkedIn/9 Выпускной проект/telecom_dataset.csv')
except:
    tariff = pd.read_csv('/datasets/telecom_clients.csv')
    data = pd.read_csv('/datasets/telecom_dataset.csv')

In [2]:
display(tariff.head())
tariff.info()

,user_id,tariff_plan,date_start
0,166713,A,2019-08-15
1,166901,A,2019-08-23
2,168527,A,2019-10-29
3,167097,A,2019-09-01
4,168193,A,2019-10-16


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      732 non-null    int64 
 1   tariff_plan  732 non-null    object
 2   date_start   732 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.3+ KB


We will not change the data format in the tariff table yet, since it is unclear how the date_start column will be used. The other columns are fine.

In [3]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              53902 non-null  int64  
 1   date                 53902 non-null  object 
 2   direction            53902 non-null  object 
 3   internal             53785 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       53902 non-null  bool   
 6   calls_count          53902 non-null  int64  
 7   call_duration        53902 non-null  int64  
 8   total_call_duration  53902 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.3+ MB


,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25


### Data preprocessing

When reviewing the data, it was clear that not all field types corresponded to the data. We will not change the data format in the tariff table yet, since it is unclear how the date_start column will be used. The other columns are fine. In data, we need to change the data type in some columns, but not everywhere there is such an opportunity. To begin with, we will remove duplicates and omissions and then return to the correction.

First, we check the data for duplicates

In [4]:
tariff.duplicated().sum()

0

In [5]:
data.duplicated().sum()

4900

There are about 9% duplicates in the dataframe data. Perhaps some of the values are really repeated, but we can't check it in any way. At the same time, given that 8 values need to be repeated at once for duplication, the probability of accidental duplication is small, so most of it was formed by an error. We will remove this data from the dataframe and, subsequently, informed colleagues about the problem

In [6]:
data = data.drop_duplicates().reset_index(drop=True)

Now let's remove the lines with an empty operator_id. It is necessary to remove them completely, since the values are qualitative, not quantitative. Accordingly, it is impossible to fill them yourself. And since we have the task of finding inefficient operators, empty operator_id strings will not be used in any way

In [7]:
data = data.query('operator_id == operator_id')
display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41546 entries, 1 to 49000
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              41546 non-null  int64  
 1   date                 41546 non-null  object 
 2   direction            41546 non-null  object 
 3   internal             41491 non-null  object 
 4   operator_id          41546 non-null  float64
 5   is_missed_call       41546 non-null  bool   
 6   calls_count          41546 non-null  int64  
 7   call_duration        41546 non-null  int64  
 8   total_call_duration  41546 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 2.9+ MB


None

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25
5,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,False,2,3,29


Of the remaining values, we see about 50 blank lines in the internal field. Potentially, this may prevent us from making a complete picture of the study, and deleting so many values will not distort the final result, so we will remove the internal nan values and get a ready dataset for the study.

In [8]:
data = data.query('internal == internal')
display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41491 entries, 1 to 49000
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              41491 non-null  int64  
 1   date                 41491 non-null  object 
 2   direction            41491 non-null  object 
 3   internal             41491 non-null  object 
 4   operator_id          41491 non-null  float64
 5   is_missed_call       41491 non-null  bool   
 6   calls_count          41491 non-null  int64  
 7   call_duration        41491 non-null  int64  
 8   total_call_duration  41491 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 2.9+ MB


None

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25
5,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,False,2,3,29


Converting data types

In [9]:
data['date'] = pd.to_datetime(data['date'])

In [10]:
data['internal'] = data['internal'].astype('bool')

In [11]:
data['operator_id'] = data['operator_id'].astype('int')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41491 entries, 1 to 49000
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype                                
---  ------               --------------  -----                                
 0   user_id              41491 non-null  int64                                
 1   date                 41491 non-null  datetime64[ns, pytz.FixedOffset(180)]
 2   direction            41491 non-null  object                               
 3   internal             41491 non-null  bool                                 
 4   operator_id          41491 non-null  int32                                
 5   is_missed_call       41491 non-null  bool                                 
 6   calls_count          41491 non-null  int64                                
 7   call_duration        41491 non-null  int64                                
 8   total_call_duration  41491 non-null  int64                                
dtypes: bool

In [13]:
data.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
1,166377,2019-08-05 00:00:00+03:00,out,True,880022,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022,True,3,0,25
5,166377,2019-08-05 00:00:00+03:00,out,False,880020,False,2,3,29


The table data is ready. Also convert the data type in the tarrif table

In [14]:
tariff['date_start'] = pd.to_datetime(tariff['date_start'])

In [15]:
tariff.head()

,user_id,tariff_plan,date_start
0,166713,A,2019-08-15
1,166901,A,2019-08-23
2,168527,A,2019-10-29
3,167097,A,2019-09-01
4,168193,A,2019-10-16


In [16]:
tariff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      732 non-null    int64         
 1   tariff_plan  732 non-null    object        
 2   date_start   732 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 17.3+ KB


The data is processed, so we can proceed to the analysis

### Data analysis

First, we will determine which call centers specialize in incoming and which ones in outgoing calls

In [17]:
data['direction'] = data['direction'].replace('out', 0)
data['direction'] = data['direction'].replace('in', 1)

In [18]:
data_prep = data.groupby('user_id').mean().query('direction < 0.5')
data_out = data.query('user_id in @data_prep.index')
data_in = data.query('user_id not in @data_prep.index')

In [19]:
data_out['user_id'].nunique()

168

In [20]:
data_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36526 entries, 1 to 49000
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype                                
---  ------               --------------  -----                                
 0   user_id              36526 non-null  int64                                
 1   date                 36526 non-null  datetime64[ns, pytz.FixedOffset(180)]
 2   direction            36526 non-null  int64                                
 3   internal             36526 non-null  bool                                 
 4   operator_id          36526 non-null  int32                                
 5   is_missed_call       36526 non-null  bool                                 
 6   calls_count          36526 non-null  int64                                
 7   call_duration        36526 non-null  int64                                
 8   total_call_duration  36526 non-null  int64                                
dtypes: bool

In [21]:
data_in['user_id'].nunique()

122

In [22]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4965 entries, 519 to 48913
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype                                
---  ------               --------------  -----                                
 0   user_id              4965 non-null   int64                                
 1   date                 4965 non-null   datetime64[ns, pytz.FixedOffset(180)]
 2   direction            4965 non-null   int64                                
 3   internal             4965 non-null   bool                                 
 4   operator_id          4965 non-null   int32                                
 5   is_missed_call       4965 non-null   bool                                 
 6   calls_count          4965 non-null   int64                                
 7   call_duration        4965 non-null   int64                                
 8   total_call_duration  4965 non-null   int64                                
dtypes: boo

The number of call centers specializing in incoming and outgoing calls does not differ as much as the number of records of the corresponding call centers. In total, the table contains 168 call centers for outgoing calls and 36526 records of their calls and 122 call centers for outgoing calls and 4965 records about them.

#### Identification of inefficient operators

##### Number of missed calls per day

Missed calls will be counted for call centers with mostly incoming traffic. We will also remove outgoing calls from the data of these call centers, since the probability of a client's response is not related to the operator's efficiency.

In [23]:
data_in_one = data_in.query('direction == 1')
data_in_missed = data_in_one.groupby('operator_id').mean().query('is_missed_call > 0.02').reset_index()
data_in_missed.head()

,operator_id,user_id,direction,internal,is_missed_call,calls_count,call_duration,total_call_duration
0,882680,166428.0,1.0,0.046512,0.069767,2.302326,142.023256,179.023256
1,888406,166669.0,1.0,0.000000,0.142857,1.285714,71.285714,107.714286
2,890582,166636.0,1.0,0.000000,0.025000,2.600000,117.375000,150.025000
3,893402,166604.0,1.0,0.074074,0.037037,4.160494,365.382716,433.679012
4,895574,166896.0,1.0,0.000000,0.021277,2.042553,238.936170,269.914894


The is_missed_call dataframe contains operator IDs with a share of unanswered calls greater than 2% and their indicators

In [24]:
data_in_missed['operator_id'].nunique()

59

There are 59 inefficient operators according to this criterion

##### Waiting time for the operator's response

In this case, as in the previous analysis, we will take for consideration call centers on the input and only incoming calls

In [25]:
data_in_waiting = data_in_one.groupby('operator_id')['total_call_duration', 'call_duration', 'calls_count'].sum().reset_index()
data_in_waiting['waiting'] = (data_in_waiting['total_call_duration'] - data_in_waiting['call_duration']) / data_in_waiting['calls_count']
data_in_waiting = data_in_waiting.query('waiting > 28')
data_in_waiting.head()

,operator_id,total_call_duration,call_duration,calls_count,waiting
0,882476,287,192,3,31.666667
5,886892,130,72,2,29.000000
7,888406,754,499,9,28.333333
13,891900,5227,3987,33,37.575758
16,891918,471,315,5,31.200000


In [26]:
data_in_waiting['operator_id'].nunique()

40

The data_in_waiting variable contains operator ids with an average waiting time of more than 28 seconds. There were 40 such operators in total

##### Operator's working hours

In [27]:
data_in_timing = data_in.groupby(['operator_id', 'date'])['total_call_duration'].sum().reset_index()
data_in_timing = data_in_timing.groupby('operator_id')['total_call_duration'].mean().reset_index()
data_in_timing = data_in_timing.query('total_call_duration < 180').reset_index(drop=True)
data_in_timing.head()

,operator_id,total_call_duration
0,882476,95.666667
1,882478,151.000000
2,884294,11.000000
3,886674,37.111111
4,886892,130.000000


The data_in_timing variable contains the ids of operators who work in call centers with mostly incoming calls and with an average conversation time of less than 3 hours per day

In [28]:
data_in_timing['operator_id'].nunique()

117

There are 117 inefficient operators according to this criterion

In [29]:
data_out_timing = data_out.groupby(['operator_id', 'date'])['total_call_duration'].sum().reset_index()
data_out_timing = data_out_timing.groupby('operator_id')['total_call_duration'].mean().reset_index()
data_out_timing = data_out_timing.query('total_call_duration < 240').reset_index(drop=True)
data_out_timing.head()

,operator_id,total_call_duration
0,880020,180.357143
1,881278,108.600000
2,883018,15.000000
3,883898,80.000000
4,884406,79.000000


The data_in_timing variable contains the ids of operators who work in call centers with mostly outgoing calls and with an average conversation time of less than 4 hours per day

In [30]:
data_out_timing['operator_id'].nunique()

241

There are 241 inefficient operators according to this criterion

The parameters studied by us helped to identify a number of inefficient operators. Nevertheless, it is too early to draw final conclusions. It may be worth paying attention to the construction of the operators' work. Also, the data may distort the principles of operation - for example, some operators may respond both to calls and in text format. Therefore, the final conclusions should be made after a more complete study of the structure and principles of operation of specific call centers. Our research will help to indicate from which call centers operators should be investigated.

### Hypothesis testing

#### The duration of outgoing and incoming calls varies

Н_0 The duration of outgoing and incoming calls is the same

Н_1 The duration of outgoing and incoming calls varies

In [31]:
data_out_duration = data_out['total_call_duration']
data_in_duration = data_in['total_call_duration']

In [32]:
from scipy import stats as st

In [33]:
results = st.ttest_ind(data_out_duration, data_in_duration, equal_var=False)
alpha = .05
print(results.pvalue)
if results.pvalue < alpha:
    print("Reject the null hypothesis")
else:
    print("Can not reject the null hypothesis")

1.2637312901343038e-123
Reject the null hypothesis


#### The number of missed calls internal and client varies

Н_0 The number of missed calls internal and client is the same

Н_1 The number of missed calls internal and client varies

In [39]:
data_internal = data[data['internal'] == True]['is_missed_call']
data_external = data[data['internal'] == False]['is_missed_call']

In [40]:
results = st.ttest_ind(data_internal, data_external, equal_var=False)
alpha = .05
print(results.pvalue)
if results.pvalue < alpha:
    print("Reject the null hypothesis")
else:
    print("Can not reject the null hypothesis")

2.385670095453404e-69
Reject the null hypothesis


#### The number of inefficient employees in call centers on tariff A and tariff B differs

In [36]:
data_tariff = data.merge(tariff, on='user_id', how='left')
data_filtered = data_tariff.query(
    '(operator_id in @data_in_missed.operator_id) or (operator_id in @data_in_waiting.operator_id) or (operator_id in @data_out_timing.operator_id)')

Н_0 The number of inefficient employees in call centers on tariff A and tariff B is the same

Н_1 The number of inefficient employees in call centers on tariff A and tariff B differs

In [41]:
data_A = data_filtered[data_filtered['tariff_plan'] == "A"]['is_missed_call']
data_B = data_filtered[data_filtered['tariff_plan'] == "B"]['is_missed_call']

In [42]:
results = st.ttest_ind(data_A, data_B, equal_var=False)
alpha = .05
print(results.pvalue)
if results.pvalue < alpha:
    print("Reject the null hypothesis")
else:
    print("Can not reject the null hypothesis")

0.018341792958485748
Reject the null hypothesis


### Conclusions

Based on the results of the study, we were able to form criteria for employee inefficiency and identify directly ineffective employees. In addition, we conducted a number of statistical studies, in which we found out that the duration of outgoing and incoming calls differs, the number of missed internal and client calls differs, and the number of non-effective employees in call centers at tariff A and tariff B is equal.